## Example of a ZVP attack on a windowed scalar multiplication algorithm

In [ ]:
from pyecsca.ec.params import get_params
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import WindowNAFMultiplier
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod
from pyecsca.ec.naf import wnaf
from pyecsca.sca.re.rpa import MultipleContext
from typing import Tuple, List

In [ ]:
# Setup the secp224r1 curve and some complete formulas and a wNAF multiplier on it with width = 5
params = get_params("secg", "secp224r1", "projective")
p = params.curve.prime
coords = params.curve.coordinate_model
add_formula = coords.formulas["add-2016-rcb"]
dbl_formula = coords.formulas["dbl-2016-rcb"]
neg_formula = coords.formulas["neg"]
mult = WindowNAFMultiplier(add=add_formula, dbl=dbl_formula, neg=neg_formula, width=5, precompute_negation=True)

In [ ]:
# Pick a "random" scalar and compute the public key
scalar = 111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111 % params.order
pubkey = params.curve.affine_multiply(params.generator.to_affine(), scalar)

In [ ]:
# A point which causes a zero to appear somewhere in the addition using the `add-2016-rcb` formula
# generated using `gen_zvp.py`
zvp_p0 = Point(coords, X=Mod(22772729384979080027796732071873389060239823427548812434098940698588, p),
                       Y=Mod(4187217282171559766870283015146241613318092832477495709411125600293, p),
                       Z=Mod(1, p))
# Multiply the ZVP point above with the inverse of `c` to obtain a point which when multiplied by `c` leads
# to the ZVP point.
def zvp_c(c):
    return params.curve.affine_multiply(zvp_p0.to_affine(), int(Mod(c, params.order).inverse())).to_model(coords, params.curve)

def query(pt: Point) -> Tuple[int, List[int]]:
    """Query the implementation and observe the ZVP side-channel,
       i.e. at which iterations a zero in the intermediate value appeared.
       Returns the total number of formula applications and indexes
       where a zero in the intermediate value appeared."""
    with local(DefaultContext()) as ctx:
        mult.init(params, pt)
        mult.multiply(scalar)
    smult, subtree = ctx.actions.get_by_index([1])
    iterations = []
    for i, formula_action in enumerate(subtree):
        for intermediate in formula_action.intermediates.values():
            values = [j.value for j in intermediate]
            if 0 in values:
                iterations.append(i)
                break
    return len(subtree), iterations

def try_guess(guess) -> bool:
    """Test if we have the right private key."""
    return params.curve.affine_multiply(g, guess) == pubkey

In [ ]:
# The attack itself, the recovered wnaf is at the bottom in `full_wnaf` after some reconstruction
# from the simulated side-channel observations.
wnaf_multiples = [1, 3, 5, 7, 9, 11, 13, 15, -1, -3, -5, -7, -9, -11, -13, -15]
all_iters = {}
for multiple in wnaf_multiples:
    rpa_point = zvp_c(multiple)
    num_iters, iters = query(rpa_point)
    all_iters[multiple] = (iters)
    print(multiple, num_iters, iters)
full = [0 for _ in range(num_iters)]
for multiple, iters in all_iters.items():
    for i in iters:
        full[i] = multiple
full_wnaf = [e for i, e in enumerate(full) if (not full[i - 1] != 0) or i in (0, 1)]
full_wnaf[0] = 1
print(full_wnaf == wnaf(scalar, 5))